In [0]:
from pyspark.sql.functions import explode,col
from pyspark.sql.functions import from_unixtime,current_timestamp

In [0]:
display(dbutils.fs.ls("/mnt/bronze-layer/2025-05-11"))

In [0]:
dbutils.widgets.text("date_val","")
date_val = dbutils.widgets.get("date_val")

In [0]:
from datetime import datetime, timedelta
if not date_val:
    yesterday_date = datetime.today() - timedelta(days=1)
    date_val = yesterday_date.strftime("%Y-%m-%d")

In [0]:
print(date_val)

In [0]:
input_path = f'mnt/bronze-layer/{date_val}/*.json'
df = spark.read.json(input_path)
df.printSchema()

In [0]:
result_df = df.select("metadata.count")
intial_count = result_df.collect()[0]['count']
display(intial_count)

In [0]:
df.select(explode('features')).display()




# df = df.withColumn('features', explode('features')).select('features.properties')
# df.display()

In [0]:
df = df.withColumn('features',explode('features')).select('features.properties')
df.display()

In [0]:
col_lst  = ['mag','place','time','updated','tz','url','detail','felt','cdi','mmi','alert','status','tsunami','sig','net','code','ids','sources','types','nst','dmin','rms','gap','magType','type','title']

for column in col_lst:
    df = df.withColumn(column,col(f'properties.{column}'))

df = df.drop('properties')

In [0]:
df.display()
print(df.printSchema())

In [0]:
final_count = df.count()

if intial_count != final_count:
    raise ValueError(f"Intial Count : {intial_count} does not match with final count {final_count}")

In [0]:
df = df.withColumn('time_timestamp',from_unixtime(col('time')/1000)).withColumn('updated_timestamp',from_unixtime(col('updated')/1000)).withColumn('ingest_ts',current_timestamp())

df.select('time','time_timestamp','updated','updated_timestamp').display()

In [0]:
df.write.mode('overwrite').format('parquet').save(f"/mnt/silver-layer/{date_val}/earthquakedata/")

In [0]:
display(dbutils.fs.ls("/mnt/silver-layer"))